In [1]:
# ==============================================================================
# FASE 2: Análise Exploratória de Dados (EDA) e Mineração (Completo)
# Arquivo: analise_fase2.py
# ==============================================================================

import os
import sys
import matplotlib.pyplot as plt
import numpy as np

# Importações Spark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, size, lower, avg, stddev, abs as _abs, round as _round, max as _max, min as _min, count
from pyspark.sql.window import Window
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.clustering import KMeans

# --- 1. Configuração de Ambiente (Windows) ---
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

print("--- Iniciando Fase 2: Análise Exploratória ---")

# --- 2. Inicializando Sessão Spark ---
spark = SparkSession.builder \
    .appName("Analise_Gastos_Fase2") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.driver.host", "127.0.0.1") \
    .master("local[*]") \
    .getOrCreate()

# Otimização Arrow
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.sparkContext.setLogLevel("WARN")

# --- 3. Carregamento dos Dados ---
BASE_DIR = os.path.join(os.getcwd(), "dados")
input_path = os.path.join(BASE_DIR, "Consolidado_Final")

print(f"📂 Buscando base consolidada em: {input_path}")

if not os.path.exists(input_path):
    print(f"❌ ARQUIVO NÃO ENCONTRADO: {input_path}")
    sys.exit() # Encerra se não achar o arquivo

try:
    df = spark.read.parquet(input_path)
    df.cache() # Cache do dataset bruto
    print(f"✅ Base carregada: {df.count()} registros.")
except Exception as e:
    print(f"❌ Erro leitura: {e}")
    sys.exit()


# ==============================================================================
# CORREÇÃO CRÍTICA: Remoção de Duplicatas
# ==============================================================================
print(f"\n--- Saneamento da Base ---")
print(f"Total Bruto: {df.count()}")

# Remove linhas onde Objeto, Valor e Favorecido são idênticos
# Isso elimina as repetições causadas pela fusão de células no Excel
df = df.dropDuplicates(['objeto_aquisicao', 'valor_transacao', 'nome_favorecido'])

# Força o recálculo e cache na memória
df.cache()
count_real = df.count()

print(f"✅ Total Real (Únicos): {count_real}")
print(f"🗑️ Lixo Removido: {54196 - count_real}")

--- Iniciando Fase 2: Análise Exploratória ---
📂 Buscando base consolidada em: c:\VSCode\projetoMineracao\dados\Consolidado_Final
✅ Base carregada: 54196 registros.

--- Saneamento da Base ---
Total Bruto: 54196
✅ Total Real (Únicos): 12572
🗑️ Lixo Removido: 41624


In [11]:
# ==============================================================================
# CÉLULA 6 (V13): NLP - Remoção de Conectivos e Termos de Ação
# ==============================================================================
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import col, size, regexp_replace, expr

print("--- Iniciando NLP V13 (Removendo 'Devido', 'Realização' e cia) ---")

# 1. Limpeza de Caracteres
# Como deve ficar (Seguro):
df_clean_chars = df.withColumn("objeto_limpo", regexp_replace(lower(col("objeto_aquisicao")), r"[^a-z]", " "))

# 2. Stopwords
stopwords_pt_custom = [
    # Artigos/Preposições/Pronomes
    "de", "a", "o", "que", "e", "do", "da", "em", "um", "para", "com", "nao", "uma", "os", "no", 
    "se", "na", "por", "mais", "as", "dos", "como", "mas", "ao", "ele", "das", "seu", "sua", "ou", 
    "quando", "muito", "nos", "ja", "eu", "tambem", "so", "pelo", "pela", "ate", "isso", "ela", 
    "entre", "depois", "sem", "mesmo", "aos", "seus", "quem", "nas", "me", "esse", "eles", "você", 
    "foi", "desta", "deste", "pelas", "pelos", "nesta", "neste", 
    
    # NOVOS VILÕES (Detectados no Raio-X do Cluster 5)
    "devido", "ser", "realizacao", "fixacao", "reposicao", "dois", "reparos", "equipamentos", 
    "prr", "covid", "bateria", # Bateria é genérico (carro? pilha?), melhor remover se for vago
    
    # Termos Genéricos / Burocracia
    "aquisicao", "referente", "pagamento", "despesa", "servico", "servicos", "material", 
    "fornecimento", "nf", "nfs", "nota", "fiscal", "cupom", "valor", "pgto", "compra", 
    "consumo", "suprimento", "fundo", "recurso", "objeto", "item", "itens", "unidade", "unid", 
    "cx", "pct", "pc", "kg", "litro", "litros", "qtd", "quantidade", "nao", "informado",
    "prestacao", "manutencao", "uso", "aplicacao", "total", "unitario", "valor",
    "atender", "solicitacao", "pregao", "ata", "registro", "preco", "conforme", "atendimento",
    "razao", "urgente", "disponivel", "utilizado", "pequeno", "grande", "novo", "velho", "aparelho", 
    "oficial", "produtos", 
    
    # Justificativas
    "acabar", "prestes", "falta", "rotina", "emergencia", "urgencia",
    
    # Instituições, Locais e CARGOS
    "pr", "rs", "prm", "dr", "dra", "sr", "sra", "ltda", "me", "epp", "sa", "co", "s/a",
    "prmcruz", "altars", "sede", "prrs", "cnpj", "cpf", "procuradoria", "empresa", "orgao",
    "procurador", "republica", "servidores", "servidor",
    "blumenau", 
    
    # Ações e Adjetivos
    "instalacao", "substituicao", "conserto", "reparo", "troca", "confeccao", "locacao",
    "gabinete", "sala", "almoxarifado", "estoque", "deposito", "setor", "unidades", "andar",
    "materiais", "diversos", "pedagio", "utilizacao", "emergencial", "seguranca", "sistema"
]

try:
    tokenizer = Tokenizer(inputCol="objeto_limpo", outputCol="words_raw")
    df_tokenized = tokenizer.transform(df_clean_chars)

    remover = StopWordsRemover(inputCol="words_raw", outputCol="words_temp")
    remover.setStopWords(stopwords_pt_custom)
    df_clean_temp = remover.transform(df_tokenized)

# FILTRO SQL (Atualizado com 'servid' e 'defeit')
    filter_expression = """
        filter(words_temp, x -> 
            x != '' AND 
            length(x) > 2 AND 
            NOT (length(x) == 4 AND substring(x, 1, 2) == 'pr') AND
            substring(x, 1, 6) != 'necess' AND
            substring(x, 1, 6) != 'demand' AND
            substring(x, 1, 7) != 'apresen' AND
            substring(x, 1, 7) != 'contrat' AND
            substring(x, 1, 5) != 'possu' AND
            substring(x, 1, 6) != 'servid' AND
            substring(x, 1, 6) != 'defeit'
        )
    """
    
    df_clean_nlp = df_clean_temp.withColumn("words_filtered", expr(filter_expression))

    df_final_nlp = df_clean_nlp.filter(size(col("words_filtered")) > 0)

    print("✅ NLP V13 concluído.")
    df_final_nlp.select("objeto_aquisicao", "words_filtered").show(5, truncate=False)

except Exception as e:
    print(f"❌ Erro NLP: {e}")

--- Iniciando NLP V13 (Removendo 'Devido', 'Realização' e cia) ---
✅ NLP V13 concluído.
+-----------------------------------------------------------------------------------------------+-----------------------------------------------------------------+
|objeto_aquisicao                                                                               |words_filtered                                                   |
+-----------------------------------------------------------------------------------------------+-----------------------------------------------------------------+
|01 agua oxigenada  10v 1 litro  \n01 detegente enzimatico 250ml \n01 filme agfa periapical c150|[agua, oxigenada, detegente, enzimatico, filme, agfa, periapical]|
|01 anel de vedacao para bacias sanitarias  com guia 1                                          |[anel, vedacao, bacias, sanitarias, guia]                        |
|01 anel de vedacao para bacias sanitarias  com guia 1                                      

In [12]:
# ==============================================================================
# CÉLULA 7 (V8 - Tuned): Vetorização Semântica (Word2Vec Ajustado)
# ==============================================================================
from pyspark.ml.feature import Word2Vec, Normalizer

print("\n--- Vetorização V8 (Word2Vec Tunado para Pequenas Bases) ---")

try:
    # AJUSTES DE HIPERPARÂMETROS:
    # vectorSize=50: Reduzi de 100 para 50 para evitar overfitting (vetores mais densos).
    # maxIter=20: Aumentei para 20 passadas sobre os dados (aprende mais).
    # windowSize=2: Janela curta para focar no produto vizinho e ignorar o departamento longe.
    # minCount=2: Baixei para 2 para capturar mais vocabulário técnico.
    
    word2Vec = Word2Vec(vectorSize=50, 
                        minCount=2, 
                        inputCol="words_filtered", 
                        outputCol="raw_features",
                        windowSize=2,
                        maxIter=20,
                        stepSize=0.025,
                        seed=42)
    
    # Treinamento
    print("Treinando modelo (isso pode levar alguns segundos a mais)...")
    model_w2v = word2Vec.fit(df_final_nlp)
    df_w2v = model_w2v.transform(df_final_nlp)
    
    # Normalização
    normalizer = Normalizer(inputCol="raw_features", outputCol="features", p=2.0)
    df_tfidf = normalizer.transform(df_w2v)
    
    df_tfidf.cache()
    print(f"✅ Vetorização Word2Vec concluída.")

except Exception as e:
    print(f"❌ Erro Word2Vec: {e}")


--- Vetorização V8 (Word2Vec Tunado para Pequenas Bases) ---
Treinando modelo (isso pode levar alguns segundos a mais)...
✅ Vetorização Word2Vec concluída.


In [13]:
# ==============================================================================
# DIAGNÓSTICO W2V: Teste de Similaridade Semântica
# ==============================================================================
print("--- Auditando a Inteligência do Modelo Word2Vec ---")

# Escolha palavras que você sabe que existem na sua base e representam grupos distintos
palavras_teste = ["chave", "torneira", "extintor", "gasolina", "limpeza", "caneta"]

try:
    for palavra in palavras_teste:
        print(f"\n🔎 Palavras mais próximas de '{palavra}':")
        
        # O método findSynonyms busca os vizinhos mais próximos no espaço vetorial
        # O segundo argumento (5) é quantas palavras queremos ver
        try:
            sinonimos = model_w2v.findSynonyms(palavra, 5)
            sinonimos.show(truncate=False)
        except Exception:
            print(f"   ⚠️ A palavra '{palavra}' não foi encontrada no vocabulário (talvez cortada pelo minCount).")

except NameError:
    print("❌ Erro: A variável 'model_w2v' não existe. Rode a Célula 7 primeiro.")

--- Auditando a Inteligência do Modelo Word2Vec ---

🔎 Palavras mais próximas de 'chave':
+---------+------------------+
|word     |similarity        |
+---------+------------------+
|chaves   |0.5950004458427429|
|gaveteiro|0.5750795602798462|
|copia    |0.5549104809761047|
|tetra    |0.5181301832199097|
|antessala|0.5160131454467773|
+---------+------------------+


🔎 Palavras mais próximas de 'torneira':
+---------+------------------+
|word     |similarity        |
+---------+------------------+
|bebedouro|0.6883059740066528|
|feminino |0.6138609647750854|
|vazamento|0.602616012096405 |
|cuba     |0.578112781047821 |
|silica   |0.569718062877655 |
+---------+------------------+


🔎 Palavras mais próximas de 'extintor':
+----------+-------------------+
|word      |similarity         |
+----------+-------------------+
|brigada   |0.5323852300643921 |
|aviso     |0.5135721564292908 |
|extintores|0.47751957178115845|
|cartazes  |0.46337205171585083|
|eletro    |0.4495670199394226 |
+---

In [16]:
# ==============================================================================
# CÉLULA 9 (V2.3): Bisecting K-Means - Correção de Vetores Vazios
# ==============================================================================
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.stat import Summarizer
from pyspark.sql.functions import col

K_FINAL = 20 

print(f"\n--- Aplicando Bisecting K-Means (k={K_FINAL}) ---")

try:
    # --- PASSO CRÍTICO: Filtrar vetores com norma zero ---
    # Usamos o Summarizer para calcular a métrica de cada vetor
    # Vetores que não têm palavras resultam em norma 0.0
    df_metrics = df_tfidf.withColumn("metrics", Summarizer.metrics("normL2").summary(col("features")))
    
    # Filtramos apenas onde a norma L2 é maior que zero
    df_input = df_metrics.filter(col("metrics.normL2")[0] > 0).drop("metrics")

    print(f"📊 Registros com conteúdo semântico: {df_input.count()}")

    # Configuração do Algoritmo
    # Cosine distance exige vetores com magnitude > 0
    bkmeans = BisectingKMeans(featuresCol="features", 
                              k=K_FINAL, 
                              seed=1, 
                              predictionCol="prediction", 
                              minDivisibleClusterSize=100,
                              distanceMeasure="cosine") 
    
    model_final = bkmeans.fit(df_input)
    df_clustered = model_final.transform(df_input)
    
    print(f"✅ Clusterização concluída com sucesso.")
    
    print("\n--- Distribuição dos Clusters ---")
    df_clustered.groupBy("prediction").count().orderBy("prediction").show(25)

except Exception as e:
    print(f"❌ Erro: {e}")


--- Aplicando Bisecting K-Means (k=20) ---
❌ Erro: [MISSING_GROUP_BY] The query does not include a GROUP BY clause. Add GROUP BY or turn it into the window functions using OVER clauses.;
Aggregate [ano#0, unidade_gestora#1, nome_suprido#2, cpf_suprido#3, periodo_aplicacao#4, aprovado#5, data_aquisicao#6, nome_favorecido#7, cpf_cnpj_favorecido#8, objeto_aquisicao#9, valor_transacao#10, ano_partition#11, objeto_limpo#4194, words_raw#4211, words_temp#4232, words_filtered#4251, raw_features#4765, features#4789, aggregate_metrics(NormL2, ComputeM2, features#4789, 1.0, 0, 0) AS metrics#5999]
+- Project [ano#0, unidade_gestora#1, nome_suprido#2, cpf_suprido#3, periodo_aplicacao#4, aprovado#5, data_aquisicao#6, nome_favorecido#7, cpf_cnpj_favorecido#8, objeto_aquisicao#9, valor_transacao#10, ano_partition#11, objeto_limpo#4194, words_raw#4211, words_temp#4232, words_filtered#4251, raw_features#4765, UDF(raw_features#4765) AS features#4789]
   +- Project [ano#0, unidade_gestora#1, nome_suprido